In [109]:
import numpy as np #creating numpy arrays
import pandas as pd #creating dataframes and storing data in dataframes
import re #for searching for text in a document
from nltk.corpus import stopwords  #removes words that don't add value to the article
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer #converts texts into numbers
from sklearn.model_selection import train_test_split #splits data into training and testing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [110]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\migue\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [173]:
new_dataset_spanish = pd.read_csv('/Users/migue/Desktop/TFM_Git/Dossier/modelo/datasets/Espanhol/dataset.csv')

new_dataset_spanish.tail()

,Id,label,Topic,Source,Headline,Text,Link
1244,568,TRUE,Covid-19,El Financiero,Encuentran nueva variante de COVID en México: ...,El Instituto de Diagnóstico y Referencia Epide...,https://www.elfinanciero.com.mx/salud/encuentr...
1245,569,FALSE,Sociedad,diariogol,El móvil de más de 60.000 euros de la princesa...,La hija del rey Felipe y de la reina Letizia y...,https://www.diariogol.com/gossip/el-movil-de-m...
1246,570,FALSE,Política,AFPFactual,NaN,"Evidentemente, Barak Obama ha sido arrestado e...","Perma | Obama, Biden y la directora de la CIA,..."
1247,571,TRUE,Covid-19,Redacción Médica,Covid: las vacunas puestas en España no alcanz...,El Ministerio de Sanidad ha actualizado los da...,https://www.redaccionmedica.com/secciones/sani...
1248,572,FALSE,Covid-19,El Sol de la Laguna,En 2014 una revista ya alertaba sobre el coron...,Fue en el mes de febrero de 2014 cuando la rev...,https://www.Noticiadelsoldelalaguna.com.mx/dob...


In [174]:
new_dataset_spanish.drop('Link', axis=1, inplace=True)
new_dataset_spanish.drop('Source', axis=1, inplace=True)
new_dataset_spanish.drop('Topic', axis=1, inplace=True)
new_dataset_spanish.drop('Headline', axis=1, inplace=True)
new_dataset_spanish.drop('Id', axis=1, inplace=True)

new_dataset_spanish.insert(0, 'Id', range(1, 1 + len(new_dataset_spanish)))

In [175]:
new_dataset_spanish.head()
new_dataset_spanish.shape[0]

1249

In [183]:
news_dataset_unico = new_dataset_spanish.replace({'True': 1, 'Fake': 0})
news_dataset_unico = new_dataset_spanish.replace({'TRUE': 1, 'FALSE': 0})
news_dataset_unico.tail()

,Id,label,Text
1244,1245,1,El Instituto de Diagnóstico y Referencia Epide...
1245,1246,0,La hija del rey Felipe y de la reina Letizia y...
1246,1247,0,"Evidentemente, Barak Obama ha sido arrestado e..."
1247,1248,1,El Ministerio de Sanidad ha actualizado los da...
1248,1249,0,Fue en el mes de febrero de 2014 cuando la rev...


In [178]:
news_dataset_unico.shape[0]

1249

In [179]:
port_stem = PorterStemmer()

def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('spanish')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [181]:
news_dataset_unico['Text'] = news_dataset_unico['Text'].apply(stemming)

In [182]:
X = news_dataset_unico.drop(columns='label', axis=1)
Y = news_dataset_unico['label']

print(X)
print(Y)

        Id                                               Text
0        1  rae incluir palabra ladi diccionario idioma es...
1        2  palabra haiga aceptada rae real academia lengu...
2        3  yordi rosado escribir dise ar nuevo libro text...
3        4  unam capacitar maestro aprobar prueba pisa m x...
4        5  alerta pretenden aprobar libro escolar conteni...
...    ...                                                ...
1244  1245  instituto diagn stico referencia epidemiol gic...
1245  1246  hija rey felip reina letizia lujo arrasa rede ...
1246  1247  evidentement barak obama sido arrestado hawai ...
1247  1248  ministerio sanidad actualizado dato evoluci n ...
1248  1249  me febrero revista interesant dedicada divulga...

[1249 rows x 2 columns]
0           0
1           0
2           0
3           1
4           0
        ...  
1244     TRUE
1245    FALSE
1246    FALSE
1247     TRUE
1248    FALSE
Name: label, Length: 1249, dtype: object


In [16]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)